<a href="https://colab.research.google.com/github/srilamaiti/pdf_summarizer/blob/main/llm_langchain_read_from_pdf_and_summarize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# https://www.youtube.com/watch?v=o11J4oO-P28
# https://www.youtube.com/watch?v=LNq_2s_H01Y
# https://colab.research.google.com/drive/1FJ7-nhTktyMSsbxI6CHye1tsfrY1Khqi?usp=sharing#scrollTo=dt7v2DVavMpX
# https://github.com/krishnaik06/Complete-Langchain-Tutorials/blob/main/Text%20summarization/summarization.ipynb

In [1]:
!pip install python-dotenv
!pip install pinecone-client
!pip install openai
!pip install langchain
!pip install -U langchain-community
!pip install langchain-openai
!pip install langchain_pinecone
!pip install langchain[docarray]
!pip install pypdf
!!pip install docarray
!pip install pydantic==1.10.8
!pip install pytube
!pip install tiktoken
!pip install ruff
!pip install PyPDF2
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.5/215.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 20.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━

In [2]:
import os
from dotenv import load_dotenv
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.callbacks import get_openai_callback
from langchain.document_loaders import PyPDFLoader
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain.docstore.document import Document
import textwrap
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
os.environ['GDRIVE_CONFIG_DIR'] = "/content/drive/MyDrive"

Mounted at /content/drive


In [3]:
!rm -rf .env
!touch .env
!cat /content/drive/MyDrive/openai_api_key.txt >> .env

In [4]:
if not os.path.exists(".env"):
    print("Error: .env file not found. Please create a .env file and set the OPENAI_API_KEY environment variable.")
    exit(1)

In [5]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
llm = OpenAI(temperature = 0)
embeddings = OpenAIEmbeddings()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [9]:
pdf_file = 'Investor-Presentation-Update_5-13-24_vf.pdf'
parser = StrOutputParser()
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")
loader = PyPDFLoader(pdf_file)
summaries = []
docs = loader.load_and_split()
chain = load_summarize_chain(llm, chain_type="map_reduce")
summary = chain.run(docs)
summaries.append(summary)
wrapped_text = textwrap.fill(summaries[0], width=100)
print(wrapped_text)

The investor presentation dated May 13, 2024, provides an overview of Stanley Black & Decker, Inc.,
focusing on tools & outdoor products, industrial products, and financial performance. The company
aims to achieve cost savings, margin expansion, and long-term growth through innovation and
operational excellence. Key points include revenue breakdown, strategic transformation plans, debt
reduction priorities, and investments in technology. The presentation also discusses the impact of
currency fluctuations, acquisitions in the lawn and garden industry, and the company's position in
the industrial engineered fastening market. Non-GAAP financial measures, liquidity sources, and
material spend breakdown are also included in the presentation.


In [10]:
chain.llm_chain.prompt.template

'Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY:'

In [11]:
chain.combine_document_chain.llm_chain.prompt.template

'Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY:'

In [12]:
chain = load_summarize_chain(llm,
                             chain_type="map_reduce",
                             verbose=True
                             )


output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary,
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"Investor 
Presentation
V e r s i o n  5 . 1 3 . 2 0 2 4"


CONCISE SUMMARY:
Prompt after formatting:
Write a concise summary of the following:


"Investor Presentation 2Contents
SWK Overview  Pages 4 – 13
Tools & Outdoor Pages 14 – 24
Industrial  Pages 25 – 29 
Appendix  Pages 30 – 37
 
Contacts
Dennis Lange
Vice President, Investor Relations
860-827-3833
dennis.lange@sbdinc.com
Christina Francis
Director, Investor Relations
860-438-3470
christina.francis@sbdinc.com
Christopher Capela
Director, Investor Relations
860-827-5556
christopher.capela@sbdinc.com
1000 Stanley Drive
New Britain, CT 06053
investorrelations@sbdinc.com"


CONCISE SUMMARY:
Prompt after formatting:
Write a concise summary of the following:


"Investor Presentation
 3Cautionary Statement
This presentation contains “forward -looking statements,” that is, statements 

In [13]:
prompt_template = """Write a concise bullet point summary of the following:


{text}


CONSCISE SUMMARY IN BULLET POINTS:"""

BULLET_POINT_PROMPT = PromptTemplate(template=prompt_template,
                        input_variables=["text"])
stuff_chain = load_summarize_chain(llm,
                             chain_type="stuff",
                             prompt=BULLET_POINT_PROMPT)

output_summary = stuff_chain.run(docs)

wrapped_text = textwrap.fill(output_summary,
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)

- Investor presentation includes SWK overview, Tools & Outdoor, Industrial, and Appendix
-
Cautionary statement regarding forward-looking statements
- Company overview with market cap,
revenue, and dividend yield information
- Geographic reach and segment breakdown
- Transformation
strategy for organic growth and margin expansion
- Strategic transformation for growth investment
and cost savings
- Value creation model focusing on branded franchises and innovation
- Balance
sheet and liquidity information
- 2024 guidance for revenue, margins, and earnings
- Focus on cash
flow, gross margin, and currency impact
- Opportunities in Tools & Outdoor, including innovation and
technology investments
- Industrial segment overview with a focus on engineered fastening
-
Reconciliation of GAAP to non-GAAP financial measures
- Material spend breakdown and liquidity
sources.


In [14]:
chain = load_summarize_chain(llm,
                             chain_type="map_reduce",
                             map_prompt=BULLET_POINT_PROMPT,
                             combine_prompt=BULLET_POINT_PROMPT)

# chain.llm_chain.prompt= BULLET_POINT_PROMPT
# chain.combine_document_chain.llm_chain.prompt= BULLET_POINT_PROMPT

output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary,
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)

- Investor presentation version 5.1 released on May 13, 2024 with updated information for investors
- Presentation includes financial data, company performance, and future projections
- Designed to
attract potential investors and showcase company's potential
- Contents include SWK Overview, Tools
& Outdoor, Industrial, and Appendix
- Forward-looking statements included, cautioning on accuracy
and referencing SEC filings
- Highlights company's streamlined structure, market cap, dividend
yield, and revenue breakdown
- Focus on geographic reach, divestiture, and transformation for growth
and cost savings
- Emphasis on margin expansion, cash generation, and currency impact
- Details on
tools & outdoor products, innovation, technology investments, and sustainability efforts
- Focus on
industrial engineered fastening industry, growth strategy, and EV market opportunities
- Appendix
provides supplementary information, charts, and tables for better understanding of main content
-
Presentation 

In [15]:
# with a custom prompt
prompt_template = """Write a concise summary of the following:


{text}


CONSCISE SUMMARY IN BULLET POINTS:"""

PROMPT = PromptTemplate(template=prompt_template,
                        input_variables=["text"])

## with intermediate steps
chain = load_summarize_chain(OpenAI(temperature=0),
                             chain_type="map_reduce",
                             return_intermediate_steps=True,
                             map_prompt=PROMPT,
                             combine_prompt=PROMPT)

output_summary = chain({"input_documents": docs}, return_only_outputs=True)
wrapped_text = textwrap.fill(output_summary['output_text'],
                             width=200,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


 

- Investor presentation for Version 5.1.13.2024 includes updates and information on company's performance and future plans, intended for potential investors.
- Presentation may include financial
data and projections, and is dated for a recent or upcoming event.
- Contents include SWK Overview, Tools & Outdoor, Industrial, and Appendix sections.
- SWK Overview covers pages 4-13, Tools &
Outdoor covers pages 14-24, and Industrial covers pages 25-29.
- Appendix section spans pages 30-37 and includes GAAP reconciliations and information on non-GAAP financial measures.
- Contacts for
Investor Relations include Dennis Lange, Christina Francis, and Christopher Capela, with phone numbers and email addresses provided.
- Company address is 1000 Stanley Drive, New Britain, CT 06053 and
email for investor relations is investorrelations@sbdinc.com.
- Presentation contains forward-looking statements addressing expected future business and financial performance, based on assumptions and
plans that

In [16]:
wrapped_text = textwrap.fill(output_summary['intermediate_steps'][2],
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)

 

- Investor presentation contains forward-looking statements 
- These statements address expected
future business and financial performance 
- They are based on assumptions and plans that may change
- Actual results may differ from projected or implied statements 
- Refer to SEC filings for
detailed information on factors that could affect results 
- Presentation does not constitute an
offer to sell or buy securities 
- Any future offering would be made under an effective registration
statement 
- Appendix includes GAAP reconciliations and information on non-GAAP financial measures.


In [17]:
chain = load_summarize_chain(llm, chain_type="refine")

output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary, width=100)
print(wrapped_text)

The existing summary provides a comprehensive overview of Stanley Black & Decker, Inc.'s investor
presentation, covering key segments such as Tools & Outdoor, Industrial, and the company's
transformation strategies for driving growth and enhancing shareholder value. The presentation
emphasizes the company's commitment to innovation, market leadership, and operational excellence, as
well as its financial flexibility, liquidity sources, and debt reduction priorities. It also
includes guidance for 2024, focusing on margin expansion, cash generation, and capital deployment
priorities. The presentation delves into the company's focus on the Tools & Outdoor segment,
including investments in technology and acquisitions in the Lawn & Garden segment. Additionally, it
discusses the Industrial segment, particularly the Engineered Fastening division, highlighting the
company's vision to be a global leader in highly engineered products with growth opportunities in
Automotive EV, Aerospace, Construc

In [18]:
prompt_template = """Write a concise summary of the following extracting the key information:


{text}


CONCISE SUMMARY:"""
PROMPT = PromptTemplate(template=prompt_template,
                        input_variables=["text"])

refine_template = (
    "Your job is to produce a final summary\n"
    "We have provided an existing summary up to a certain point: {existing_answer}\n"
    "We have the opportunity to refine the existing summary"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the original summary"
    "If the context isn't useful, return the original summary."
)
refine_prompt = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template,
)
chain = load_summarize_chain(OpenAI(temperature=0),
                             chain_type="refine",
                             return_intermediate_steps=True,
                             question_prompt=PROMPT,
                             refine_prompt=refine_prompt)

In [19]:
output_summary = chain({"input_documents": docs}, return_only_outputs=True)
wrapped_text = textwrap.fill(output_summary['output_text'],
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)



SWK, a leading global manufacturer of tools and outdoor equipment, is undergoing a strategic
transformation led by Vice President Dennis Lange to accelerate organic growth. The Investor
Relations team is available to assist with any inquiries. The Investor Presentation Version
5.1.13.2024 highlights SWK's streamlined company structure, strong market cap and dividend yield,
diverse portfolio of iconic brands, powerful innovation, broad category and channel coverage,
operational excellence, and global reach. The company aims to reduce complexity and achieve ~$2B in
savings by 2025, with a focus on core growth and enhancing shareholder return. This includes a
supply chain transformation and targeting $1.5 billion in pre-tax run-rate savings by the end of
2024. Investors are advised to refer to the company's SEC filings for detailed information. SWK's
value creation model includes world-class branded franchises with sustainable strategic
characteristics, strong innovation-driven business

In [20]:
wrapped_text = textwrap.fill(output_summary['intermediate_steps'][0],
                             width=100,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)


The Investor Presentation Version 5.1.13.2024 is a document that contains key information for
investors.
